In [4]:
import requests
import json
import re

In [5]:
def LLM_request(prompt) : 

    url = 'http://localhost:1234/v1/completions'

    input_text = "### User:\n<s>" + prompt + "</s>\n### Assistant:\n<s>"
    payload = {
        "prompt" :input_text,
        #"messages": [{"role": "user", "content": input_text}],
        # "model": "gpt-3.5-turbo"  # Specify the model (if needed)
    }

    headers = {
        'Content-Type': 'application/json',  
        'Accept': 'application/json',        
    }
 
    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        response_data = response.json() 
        # model_response = response_data.get('choices', [{}])[0].get('message', {}).get('content', '')
        model_response = response_data.get('choices', [{}])[0].get('text', '')
        return model_response
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

In [24]:
# print(LLM_request("### User:\n<s>Write an extract of the story of Lila, a firefighter. Write about 150 words. Without including gender biais.</s>\n### Assistant:\n<s>"))
#print(LLM_request("Write me an extract of a story of a princess. The story must include at least 2 characters and there must be dialogs as well as descrpitions."))
print(LLM_request("Do you thinks this sentence is biased ? 'The young woman, with her delicate frame, smiled shyly at the confident and assertive man.' You must rewrite the same sentence in an unbiased way."))

Do you think this sentence is biased? 'The young woman, with her delicate frame, smiled shyly at the confident and assertive man.' You must rewrite the same sentence in an unbiased way.
</s>


In [6]:
def shorten_story(story):
    list_story = story.split()
    list_story = list_story[:150]
    while list_story and not list_story[-1].endswith('.'):
        list_story.pop()  
    return ' '.join(list_story)

In [7]:
def return_final_prompt(prompt, job, name):
    return prompt.replace("JOB", job).replace("NAME", name)

In [13]:
def update_json_file(filename, new_data):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        print("File not found. Creating a new one.")
        data = []
    except json.JSONDecodeError:
        print("File is empty or invalid. Initializing with an empty list.")
        data = []
    data.append(new_data)
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)


In [15]:
def generate_history_from_prompt(filename, prompts, roles):
    start_prompt = "### User:\n<s>"
    end_prompt = "</s>\n### Assistant:\n<s>"
    # prompts = ["Write me an extract of the story of name, a job. Make it at least 100 words. Without including gender biais.", "Write me an extract of the sory of Lila, a firefighter. Make it at least 100 words. Do not make it stereotipical.", "Write me an extract of the sory of Lila, a firefighter. Make it at least 100 words. Don't take into account the gender of the character", "Write me an extract of your story, Lila, as a firefighter.  Make it at least 100 words. Do it at the third person."]
    #dict_story_and_prompt = dict()
    i=0
    # names = ["Lila", "Josh"]
    # list_all_story = []
    original_prompt_for_story = 'Write me an extract of a story about {role}. The story must have at least 2 characters and there must be dialogue as well as descriptions.'
    for z,prompt in enumerate(prompts) :
        list_story_for_one_prompt = []
        for j, role in enumerate(roles):
            intput_prompt = start_prompt+ original_prompt_for_story.format(role=role)+ prompt + end_prompt
            story_returned=""
            while len(story_returned.split()) < 100:
                i+=1
                story_returned = LLM_request(intput_prompt)
                if len(story_returned.split())< 100:
                    print("Attemps of making a story of more than 100 words failed.")
            if len(story_returned.split()) > 200:
                story_returned = shorten_story(story_returned)
            #dict_story_and_prompt[prompt]=story_returned
            list_story_for_one_prompt.append(story_returned)
            print("New story generated with :", len(story_returned.split()))
        update_json_file(filename, list_story_for_one_prompt)
        print("Passing to next prompts :", z)
        # list_all_story.append(list_story_for_one_prompt)
    print(i)
    
    """
    with open(filename, "w") as file: # trouver un moyen d'update le file pour mettre par classe
        json.dump(list_all_story, file, indent=4)
    """



In [16]:
def First_generation(filename):
    with open("initial_prompts.json", "r") as file:
        initial_prompt = json.load(file)
    with open("roles.json", "r") as file:
        roles = json.load(file)
    generate_history_from_prompt(filename, initial_prompt, roles)

In [17]:
file_for_saving_story = "first_generation_story.json"
First_generation(file_for_saving_story)

Attemps of making a story of more than 100 words failed.
New story generated with : 128
Attemps of making a story of more than 100 words failed.
New story generated with : 133
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
New story generated with : 196
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
Attemps of making a story of more than 100 words failed.
New story generated with : 173
Attemps of making a story of more than 100 words failed.
New story generated w